In this file, we prompt the user for a string, and a given alphabet size. What is output is the number of keystrokes, on average, it would take for the string to appear in a random sequence of letters belonging to the alphabet with the given size. So, for example, if the string we are looking for is 'ABCBA', and we have an alphabet of size 20 (so we will have a random sequence of all letters of the alphabet, except for the last 6), it will take, on average, 3200020 keystrokes to see the string 'ABCBA' appear.

What is interesting about this demo is that not all strings have the same expected time, even if the alphabet size and the two strings' lengths remain constant. This is because of a very slick argument that is described in this article below: https://www.theguardian.com/science/2023/mar/20/did-you-solve-it-the-infinite-monkey-theorem

The above article uses the underlying Markov Chains to illustrate this concept. What we found, however, is that there is a bijection between the number of keystrokes it takes (or in the case of the article above, the number of seconds, on average, it takes a monkey to type a given string when typing 1 random character each second) and the number of bets that must be placed, when bettors are using the Martingale Betting approach to make wagers, to expect a net payout of 0 between all bettors betting on which letter will appear next in the random sequence of letters. This argument is explained in the article below:
https://martingalemeasure.wordpress.com/2014/02/02/monkey-typing-abracadabra-14/

This code file allows users to input an alphabet size, and a string, and will output the average number of keystrokes it will take. The core of this calculation lays in the generation of a word's prefixes. What we found was, the only way that the average keystrokes was NOT equal to $AlphabetSize^{StringLength}$ is if the string ends in any substring of itself, regardless of how many substrings are included (for example, "ABRACADABRA" ends in 2 prefixes; "ABRA", and "A"). We should note that, as of 9/7/2023, the users were allowed to add custom odds for each letter in their alphabet occurring to allow for more dynamic calculations.

On 11/12/2023, another major implementation was added, where we allow users to input any string of letters, and then a transition matrix is returned. This is useful for also calculating the hitting time's Probability Mass Function (PMF)

On 2/16/2024, a major implementation was added, where an algorithm that calculates the probability that one string occurs before another was implemented. This is referred to as the "two player game", where two users can both input a string to see which will be expected to occur first. This is also extended to allow for any number of players (and consequently, any number of target strings).

In [1]:
import math
import string
from fractions import Fraction
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


#Turn Scientific off
np.set_printoptions(suppress=True,
   formatter={'float_kind':'{:f}'.format})

### Prompt user for alphabet size function

**input**: NA  
**output**: NA  

This function prompts the user for a size of an alphabet. This starts at A, then B, then C, etc... until the desired size is met.

In [2]:
def prompt_for_alphabet():
    #Initial condition
    alphabet_size = 0
    
    #While there is an invalid alphabet size...
    while alphabet_size < 1 or alphabet_size > 26:
        
        #Prompt the user for an alphabet size
        alphabet_size = int(input('Please input alphabet size (1-26): '))
        
        #If it is a valid size, save the input, return the input
        if alphabet_size >= 1 and alphabet_size <= 26:
            return alphabet_size
        
        #If it is an invalid size, print error message, thus iterating thru the while again
        else:
            print('Invalid alphabet size. Please try again.')

### Prompt user for string function

**input**:  
*alphabet_size*: The length of the alphabet specified in the prompt_for_alphabet() function.  
**output**: NA  

This function prompts the user for a target string. Its number of unique characters must not exceed the length of the alphabet.

In [3]:
def prompt_for_string(alphabet_size):
    
    #Initial condition. This will allow the loop to repeat until a valid string is entered
    valid_string = False
    
    while ~valid_string:
        user_input = input("Please input a string of characters. Note that the number of unique characters cannot exceed the size of the alphabet: ").upper()
        
        #We want to count the number of unique chars in the input. We do this below:
        user_input_set = set(user_input)
        num_unique = len(user_input_set)
        
        #Used to check the "elif" case below
        alphabet = list(string.ascii_uppercase)
        
        #Handle invalid input: if the number of unique letters in the string is greater than the alphabet
        if num_unique > alphabet_size:
            print('Invalid string given your alphabet size. Please try again.\n')
        
        #We also want to check to make sure that the string is valid, in the sense that, for example, if there is an 
        #alphabet of size 2, than only As and Bs are in the string. For this, we will need to use the list of letters
        #from the string library
        elif not user_input_set.issubset(set(alphabet[0:alphabet_size])):
            print(f'Invalid string. Given your alphabet size of {alphabet_size}, please let your string contain only the first {alphabet_size} letters of the alphabet.\n')
            
        
        #Handle "good" input, where we return the input.
        else:
            return user_input

### Get prefixes function

**input**:  
*input_string*: The target string specified from the prompt_for_string(alphabet_size) function.  
**output**:  
*prefixes*: A list of the input_string's prefixes.  

This function returns the prefixes of an input_string. It is a list, of length len(input_string+1), where each index, i, of the prefixes list contains the first i characters of input_string.

In [4]:
def get_prefixes(input_string):
    
    #I want to get the prefixes of the string
    prefixes = []
    

    #Loop thru the string and get the prefixes
    for i in range(len(input_string)+1):
        prefixes.append(input_string[0:i])
    return prefixes

### Get lengths of the prefixes that appear at the end of the string. 

**input**:  
*input_string*: The target string specified from the prompt_for_string(alphabet_size) function.  
*prefixes*: The list of prefixes specified from the get_prefixes(input_string) function.  
**output**:  
*lengths_to_return*: The lengths of the prefixes that appear at the end of the input string. This is used when calculating the average hitting time of the input_string.  

In [5]:
def get_lengths_of_prefixes(input_string, prefixes):
    
    #Initialize the lengths of the prefixes that appear list. We fill it in the for loop below
    lengths_to_return = []
    
    #Loop thru each prefix
    for prefix in prefixes:
        
        #if the prefix equals the last prefix_length chars in the string, we add the string to the list to return
        if input_string[-len(prefix):] == prefix:
            lengths_to_return.append(len(prefix))
    return lengths_to_return

### Specify the odds of each letter in the alphabet appearing

**input**:  
*alphabet_size*: The length of the alphabet specified in the prompt_for_alphabet() function.   
**output**:  
*odds*: A dictionary mapping each letter in the alphabet to its probability of appearing in the random string of characters.  

In this function, we prompt users for the odds of each letter from their alphabet occurring in a random string of characters. Note that this function ensures valid input, and provides the users with a default option of each letter having an equal probability.

In [6]:
def specify_odds(alphabet_size):
    #Get list of alphabet. We can iterate thru this to get the odds
    alphabet = list(string.ascii_uppercase)
    
    #Specify empty dict to hold the letters and the associated odds
    odds = {}
    
    #Lets ask the user if the odds are all even. If so, this will be easy to fill and we do not need them to do it.
    even_odds_input = input('Would you like each letter to have equal odds of occurring? Please reply Y or N: ')
    
    #If the user wants each letter to have an even chance to occur, we handle that here, where we will and return the odds
    #dict accordingly.
    if even_odds_input.lower() == 'y':
        #Add the letters in the alphabet and the even odds to the dict
        for letter_index in range(alphabet_size):
            odds[alphabet[letter_index]] = 1/alphabet_size
    
    #If the user does not want each letter to have an even chance to occur, we handle that here, and prompt for each 
    #chance invidividually, while also making sure that there are no 0s and the odds add to 1.
    elif even_odds_input.lower() == 'n':
        for letter_index in range(alphabet_size):
            
            #If they enter this in fraction form, we want to be able to handle it, which is why we typecast it to a Fraction
            current_odds = Fraction(input(f'Please input odds for letter {alphabet[letter_index]}: '))
            #Then, we typecast to a float
            current_odds = float(current_odds)
            
            #Handle bad input
            if current_odds == 0:
                return "Please try again. We cannot have a 0 for the probability of a letter occuring."
            
            #Set the odds of the current letter
            odds[alphabet[letter_index]] = current_odds
            
        #Need to ensure that all odds add to 1
        if round(sum(odds.values()), 5) != 1:
            return "Please try again and ensure that all odds add to 1"
        
    #If they enter anything other than Y or N, return an error message.
    else:
        return "Invalid Input. Please Try Again"
    return odds 

### Calculate average time function

**input**:  
*alphabet_size*: The length of the alphabet specified in the prompt_for_alphabet() function.  
*input_string*: The target string specified from the prompt_for_string(alphabet_size) function.  
*odds*: The odds dictionary specified from the above specify_odds(alphabet_size) function.  
**output**:    
*avg_time*: Given our parameters, the average number of keystrokes that the input_string will occur.   

We use the Martingale article to calculate the average time here. This is a more time-efficient way of calculating the average time (opposed to iterating through the hitting time PMF calculated later in this code).

In [7]:
def calculate_average_time(alphabet_size, input_string, odds):
    
    if type(odds) == str: #If an error messgae was returned, immediately break out of this function.
        return odds
    
    #Get a list of the prefixes of the string
    prefixes = get_prefixes(input_string)
    
    #Get a list of the lengths of prefixes that appear at the end of the string
    lengths = get_lengths_of_prefixes(input_string, prefixes)
    print(lengths)
    
    #Calculate the average time.    
    #In this calculation, we now need to add the product of the recip. of each of the corresponding odds of the letters
    #in the prefixes that are found.
    
    avg_time = 0
    for length in lengths:
        
        #We set this to 1 because we are multiplying
        to_add = 1
        for letter in prefixes[length]:
            to_add*=(1/(float(odds[letter])))
            print(to_add)
        avg_time+=to_add
        
        
    return avg_time

### Build Transition Matrix 

**input**:  
*alphabet*: The length of the alphabet specified in the prompt_for_alphabet() function.  
*string*: The target string specified from the prompt_for_string(alphabet_size) function.  
*odds*: The odds dictionary specified from the above specify_odds(alphabet_size) function.  
**output**:    
*transition_matrix*: The associated parameters' transition matrix.

Here, we calculate a transition matrix for any input string, alphabet, and odds dictionary. This is for a *one player game*, or one string, opposed to two, which is later in this code.

In [8]:
def build_transition_matrix_one_string(string, alphabet, odds):
    
    #Initialize a transition matrix of all zeros. We will add to this 1 row at a time in the below for loops.
    transition_matrix = np.zeros(((len(string)+1), (len(string)+1)))
    
    #Get the prefixes of the string
    prefixes = get_prefixes(string)
    
    #Iterate thru the string
    for index_of_current_letter in range(len(string)):
        #Iterate thru alphabet
        for index_of_current_alphabet_letter in range(len(alphabet)):
            
            #Compare the current letter of the string to the current letter of the alphabet. if they are the same, 
            #we increment the given value of the transition matrix by the probability of that letter occurring.
            if string[index_of_current_letter] == alphabet[index_of_current_alphabet_letter]:
                transition_matrix[index_of_current_letter][index_of_current_letter+1]+=odds[alphabet[index_of_current_alphabet_letter]]
            
            #If they are not equal, we must perform something a bit more complicated, where we find the longest prefix 
            #for the first index_of_current_letter chars in the string + the character 
            #alphabet[index_of_current_alphabet_letter] (the mismatched character) that is left off on, so we know what
            #part of the Markov Chain we return back to.
            else:
                
                #Initialize the temp string: the string from 0 to the current letter, plus the mismatched (current) char
                #of the alphabet.
                temp_string = string[0:index_of_current_letter]+alphabet[index_of_current_alphabet_letter]
                
                #Initialize the longest prefix to 0
                longest_prefix = 0
                
                #Iterate thru the temp string and find the longest prefix.
                for index_of_letter_in_temp_string in range(len(temp_string)):
                    
                    #if the last index_of_letter_in_temp_string chars are a prefix, we update the longest prefix.
                    if temp_string[-index_of_letter_in_temp_string:] in prefixes:
                        longest_prefix = len(temp_string[0:index_of_letter_in_temp_string])
                
                #Uodate the corresponding entry of the transition matrix
                transition_matrix[index_of_current_letter][longest_prefix]+=odds[alphabet[index_of_current_alphabet_letter]]
                
    #Because we are dealing with an absorbing Markov Chain, we set the bottom-right entry to 1.
    transition_matrix[len(string)][len(string)]+=1
    
    #Return the transition matrix
    return transition_matrix
                                                                        

### Calculate Hitting Time PMF Function

**input**:  
*PHat*: THe transition matrix derived from the above build_transition_matrix_one_string(string, alphabet, odds) function.  
*string*: The target string specified from the prompt_for_string(alphabet_size) function.  
*stopping_prob=.00001*: A stopping condition for the function that is met (we need this because of the while loop) when the probability that was just calculated is less than the stopping_prob, and the one calculated before is greater. This is to ensure it does not terminate for a low value.  

**output**:    
*probs_list*: The Hitting Time PMF.   

Here, we calculate the Hitting Time's PMF. Note that summing the product of each term * its index will allow us to approximate the average hitting time.

In [9]:
def calc_hitting_time_PMF(PHat, string, stopping_prob=.00001):
    
    #if the string is empty, return 1
    if len(string) == 0:
        return [1]
    
    #Initialize a dict that the probabilities can go into
    probs_list = {}
    
    #Fill the first element of the probs_list. This will always be 0.
    probs_list[0] = 0
    
    #e1 is the first row of an NxN identity matrix (N = word length + 1)
    #et is the last column of an NxN identity matrix
    
    identity = np.identity(len(string)+1)
    
    e1 = identity[0]
    et = identity[:,[len(string)]]
    
    #Create a variable that lets the while loop run and calculate the probabilities of the hitting time being X.
    #perform_calculations will be set to False when the probability is less than the stopping_prob and the probability just 
    #calculated is less than the one we just calculated (to avoid a case where we halt the while loop when the prob = 0)
    perform_calculations = True
    
    #Create an index for the hitting time we are currently calculating the probability for
    X = 1
    
    while perform_calculations:
        #Intermediate step: Raise matrix to power X-1
        PHat_exp = np.linalg.matrix_power(PHat, X-1)
    
        #Intermediate step: PHat - I
        PHat_minus_I = PHat - identity
        
        #Calculate the probability
        prob = e1 @ PHat_exp @ PHat_minus_I @ et
        
        #Add the probability to the probs list
        probs_list[X] = float(str(prob[0]))
        #Check to see if we need to make the perform_calculations False. Reads as...
        
        #If the prob we just calculated is LESS than the one we calculated in the previous iteration, AND the prob we just
        #calculated is less than the stopping_prob, set perform_calculations to False
        if (probs_list[X] < probs_list[X-1]) and probs_list[X] < stopping_prob:
            perform_calculations = False
        
        #Increment X
        X+=1
    return probs_list

### Get Suffixes Function

**input**:  
*input_string*: The target string specified from the prompt_for_string(alphabet_size) function.  
**output**:  
*suffixes*: A list of the input_string's suffixes.  

This function returns the suffixes of an input_string. It is a list, of length len(input_string+1), where each index, i, of the prefixes list contains the last i characters of input_string.

In [10]:
def get_suffixes(input_string):
    
    #I want to get the suffixes of the string
    suffixes = []
    

    #Loop thru the string and get the suffixes
    for i in range(len(input_string)+1):
        suffixes.append(input_string[i:])
    return suffixes

### Get all Valid States Function

**input**:  
*string1*: The first string entered by the user  
*string2*: The second string entered by the user  
**output**:  
*sorted(list(set(all_valid_states_list)), key=lambda x: len(x))*: A list of the valid states for the two-player game 

A few things to define... 
- "State": A state, in this two-player game, are all (len(string1)+1)$*$(len(string2)+1) combinations of string1 and string2's prefixes. This includes empty strings. They are formatted as "prefix1,prefix2"
-  "Valid State": A valid state is a state that the two-player Markov chain can actually go to, based on what the inputs are. Most of these can never be reached, and it is easier to filter them out here.

In [11]:
def get_all_valid_states(string1, string2):
    
    #Get string1 and string2's prefixes
    string1_prefixes = get_prefixes(string1)
    string2_prefixes = get_prefixes(string2)
    
    #Init. an empty list of all valid states
    all_valid_states_list = []
    
    #Iterate through the indices of string1...we do this to see, as we progress through string1, how are we affecting the
    #progression through string2?
    for letter_index in range(len(string1)+1):
        
        #Take the prefix of string1 of length letter_index
        substring_of_string1 = string1[0:letter_index]
        
        #Obtain all suffixes of this prefix
        substring_of_string1_suffixes = get_suffixes(substring_of_string1)
        
        #Get intersection of string2's prefixes with these substring's suffixes
        temp_intersection = list(set(substring_of_string1_suffixes).intersection(string2_prefixes))
        
        #If there was an intersection, we add this to the list of valid states with the formatting convention listed above
        if len(temp_intersection) > 0:
            all_valid_states_list.append(string1[0:letter_index] + ',' + str(max(temp_intersection, key=len)))
            
    #Iterate through the indices of string2...we do this to see, as we progress through string2, how are we affecting the
    #progression through string1?      
    for letter_index in range(len(string2)+1):
        
        #Take the prefix of string2 of length letter_index
        substring_of_string2 = string2[0:letter_index]
        
        #Obtain all suffixes of this prefix
        substring_of_string2_suffixes = get_suffixes(substring_of_string2)
        
        #Get intersection of string1's prefixes with these substring's suffixes
        temp_intersection = list(set(substring_of_string2_suffixes).intersection(string1_prefixes))
        
        #If there was an intersection, we add this to the list of valid states with the formatting convention listed above
        if len(temp_intersection) > 0:
            all_valid_states_list.append(str(max(temp_intersection, key=len)) + ',' + string2[0:letter_index])
        
    #Return this list, sorted by length so that the absorbing states are at the end.
    return sorted(list(set(all_valid_states_list)), key=lambda x: len(x))

### Initialize df function

**input**:  
*all_valid_states_list*: The list of valid states derived from the above function  
**output**:  
*states_df*: An empty transition matrix with the rows and columns named for simplicity.   

Although this df is turned into a np.array later, it was easier to visualize what was going on here by naming the rows and columns in a pd.DataFrame.

In [12]:
def initialize_df(all_valid_states_list):
    
    #Intialize an empty df with rows and columns named as the valid states
    states_df  = pd.DataFrame(columns=all_valid_states_list, index=all_valid_states_list)
    
    #Set every entry to 0 instead of NaN, so we can do += on the cell values later.
    states_df.loc[:,:] = 0
    
    #Return the states df
    return states_df

### Make the two-player Transition Matrix (DataFrame)

**input**:  
*string1*: The first string entered by the user    
*string2*: The second string entered by the user     
*alphabet*: The length of the alphabet specified in the prompt_for_alphabet() function.  
*odds*: The odds dictionary specified from the above specify_odds(alphabet_size) function.  
*states_df*: An empty transition matrix with the rows and columns named for simplicity, generated from the above function.  

**output**:  
*to_return_df*: A dataframe representing the two-player game's transition matrix.

In [13]:
def make_two_player_transition_df(string1, string2, alphabet, odds_dict, states_df):
    
    #Copy the input df. I dont want to change it directly, so we make a copy of it
    to_return_df = states_df.copy()
    
    #Get a list of all of the states. We iterate thru this
    all_states_list = list(states_df.index)

    #Get the prefixes of the 2 input strings
    string1_prefixes = get_prefixes(string1)
    string2_prefixes = get_prefixes(string2)
    
    #Iterate thru the states
    for state in all_states_list:
        
        #Turn the 2 states  that make up each state name into their own state names
        state1, state2 = state.split(',')
        
        #Make into list so we can iterate thru these
        temp_states_list = [state1, state2]
        
        #I want to see if either of these should be absorbing. We check that here to avoid unecessary calc.'s
        
        #Do the strings tie? (Goes first in case of short circuit)
        if len(state1) == len(string1) and len(state2) == len(string2):
            to_return_df.at[state, state]
            
        #Is string 1 completed first?
        elif len(state1) == len(string1):
            to_return_df.at[state, state]+=1
    
        #Is string 2 completed first?
        elif len(state2) == len(string2):
            to_return_df.at[state, state]+=1
            
        else:

            #Get the current string, which is the max of whatever subset state name (1 or 2) we are in
            current_string = max(temp_states_list, key=len)
            
            #Iterate thru the alphabet. Notice the similarities from the 1 string function?
            for letter in alphabet:
                
                #Create a temp string: What WOULD the last few chars look like if the letter was next?
                temp_string = current_string + letter
                
                #Get the suffixes of this
                temp_string_suffixes = get_suffixes(temp_string)

                #Find the intersections with string1 and string2's prefixes with these suffixes. This will allow us to 
                #update what state we go to 
                temp_intersection_with_string1_prefs = list(set(temp_string_suffixes).intersection(string1_prefixes))
                temp_intersection_with_string2_prefs = list(set(temp_string_suffixes).intersection(string2_prefixes))

                #Create empty strings that are filled depending on what state we will go to next
                temp_string_intersection_with_string1_prefs = ''
                temp_string_intersection_with_string2_prefs = ''

                #Convert these intersections to lists
                if len(temp_intersection_with_string1_prefs) > 1:
                    temp_string_intersection_with_string1_prefs+=str(max(temp_intersection_with_string1_prefs, key=len))

                if len(temp_intersection_with_string2_prefs) > 1:
                    temp_string_intersection_with_string2_prefs+=str(max(temp_intersection_with_string2_prefs, key=len))
                
                #Format the go_to_state, the state we would go to if the letter were to appear next
                go_to_state = temp_string_intersection_with_string1_prefs+','+temp_string_intersection_with_string2_prefs
                
                #Add the odds of this letter occurring to the proper spot of the to_return_df
                to_return_df.at[state, go_to_state]+=odds_dict[letter]

    return to_return_df

### Perform the two player game by seeing the probability of string1 or string2 occurring first

**input**:  
*two_player_df*: The transition matrix derived from the above function 
*num_keystrokes*: How many keystrokes will the game go until?

**output**:  
*init_dist @ PHat_exp*: Probability distribution of the two-player transition matrix after num_keystrokes_letters

In [14]:
def two_player_game(two_player_df, num_keystrokes):
    
    #Get the np.array of the transition df 
    PHat = two_player_df.values
    
    #Get the intiial dist from the associated identity matrix of the same size as the transition df
    identity = np.identity(two_player_df.shape[0])
    init_dist = identity[0]

    
    #Phat^num_keystrokes: Intermediate step for the probability distribution calculation
    PHat_exp = np.linalg.matrix_power(PHat, num_keystrokes)
    
    #Return the probability distribution
    return init_dist @ PHat_exp

# N-Player Game

In the following secctions, code is implemented to perform the "N-Player Game", which is an extension of the two player game. The two player code above can technically be achieved through this N-Player game for N=2, however, to document the work that was done for this project, we leave in the two-player game material.

### Get all Valid States for N-Player Game Function

**input**:  
*string_list*: A list of input strings  
**output**:  
*sorted(list(set(all_valid_states_list)), key=lambda x: len(x))*: A list of the valid states for the N-player game 

In [ ]:
def get_all_valid_states_n_players(string_list):
    
    #Init. prefixes list that we fill with all input strings' prefixes
    prefixes_list = []
    
    #Get all strings' prefixes
    for player_string in string_list:
        prefixes_list.append(get_prefixes(player_string))
    
    #Init. an empty list of all valid states
    all_valid_states_list = []
    
    #Now, instead of iterating through each of the 2 input strings seperately, we need to iterate thru all input strings
    #together and repeat the same processes as the 2-player code
    for player_string in string_list:
        
        #We will need this later
        player_string_index = string_list.index(player_string)
        
        #Iterate through the indices of player_string...we do this to see, as we progress through player_string, 
        #how are we affecting the progression through the other input strings?
        for letter_index in range(len(player_string)+1):
            
        
            #Take the prefix of player_string of length letter_index
            substring_of_player_string = player_string[0:letter_index]
        
            #Obtain all suffixes of this prefix
            substring_of_player_string_suffixes = get_suffixes(substring_of_player_string)
        
            #Previously, we got the intersection of string2's prefixes with the substring's suffixes. 
            #Well, instead of just doing this for string2, we now need to do it for all of the input strings...
            #for this, we need another for loop, iterating through all of the indices of string_list that do not 
            #include the player_string's index.
        
            #Make a list of the indices to iterate thru
            to_iterate = []
            to_iterate = list(range(0, len(string_list)))
            to_iterate.remove(player_string_index)
        
            #Init. empty list that we fill with the valid states.
            valid_states = []
        
            for other_index in to_iterate:
                #Get intersection of string_list[other_index] prefixes with these substring's suffixes
                temp_intersection = list(set(substring_of_player_string_suffixes).intersection(prefixes_list[other_index]))
        
                #If there was an intersection, we add this to the list of valid states with the formatting convention 
                #listed above
                if len(temp_intersection) > 0:
                    valid_states.append(str(max(temp_intersection, key=len)))
                
                #If there was instead not an intersection, we add a blank string.    
                else:
                    valid_states.append('')
        
            #Lastly, we need to set valid_states[player_string_index] to substring_of_player_string.
            valid_states.insert(player_string_index, substring_of_player_string)
        
            #Add the contents of valid_states to all_valid_states_list
            all_valid_states_list.append(','.join(valid_states))
    
    #There is a problem with this list. If a target string is a prefix of another target string, this function will return more valid states than desired. This is handled here.
    
    #There are duplicates in this list. Lets remove them when returning
    
    #Return this list, sorted by length so that the absorbing states are at the end.
    return sorted(list(set(all_valid_states_list)), key=lambda x: len(x))

### Make the two-player Transition Matrix (DataFrame)

**input**:  
*string_list*: The input list of strings    
*alphabet*: The length of the alphabet specified in the prompt_for_alphabet() function.  
*odds*: The odds dictionary specified from the above specify_odds(alphabet_size) function.  
*states_df*: An empty transition matrix with the rows and columns named for simplicity, generated from the above function.  

**output**:  
*to_return_df*: A dataframe representing the N-player game's transition matrix.

In [ ]:
def make_n_player_transition_df(string_list, alphabet, odds_dict, states_df):
    
    #Copy the input df. I dont want to change it directly, so we make a copy of it
    to_return_df = states_df.copy()
    
    #Get a list of all of the states. We iterate thru this
    all_states_list = list(states_df.index)

    #Init. prefixes list that we fill with all input strings' prefixes
    prefixes_list = []
    
    #Get all strings' prefixes
    for player_string in string_list:
        prefixes_list.append(get_prefixes(player_string))
    
    #Iterate thru the states
    for state in all_states_list:
        
        #Turn the states that make up each state name into their own state names
        temp_states_list = state.split(',')
        
        #I want to see if either of these should be absorbing. We check that here to avoid unecessary calc.'s
        #This is similar to the two player case, but we see if there is any instance where the length of the state name
        #at an index = the length of the string at the same index.
        
        #I also want to see if this is an absorbing state. This lets me bypass the bottom code:
        absorbing_state = False
        
        for index_of_state in range(len(temp_states_list)):

            if len(string_list[index_of_state]) == len(temp_states_list[index_of_state]):
                to_return_df.at[state, state]=1
                absorbing_state = True

        if not absorbing_state:

            #Get the current string, which is the max of whatever subset state name (1 or 2) we are in
            current_string = max(temp_states_list, key=len)
            
            #Iterate thru the alphabet. Notice the similarities from the 1 string function?
            for letter in alphabet:
                
                #Create a temp string: What WOULD the last few chars look like if the letter was next?
                temp_string = current_string + letter
                
                #Get the suffixes of this
                temp_string_suffixes = get_suffixes(temp_string)

                #Find the intersections with each strings' prefixes with these suffixes. This will allow us to 
                #update what state we go to 
                
                #Create temp intersection list, then fill it
                temp_intersection_list = []
                
                for string in string_list:
                    temp_intersection_list.append(
                        list(set(temp_string_suffixes).intersection(prefixes_list[string_list.index(string)])))

                #Create empty strings list that are filled depending on what state we will go to next
                temp_strings_intersection_list = ['']*len(string_list)
                
                #Iterate thru temp intersection list, converting these intersections to lists.
                for index_of_intersection in range(len(temp_intersection_list)):
                    
                    if len(temp_intersection_list[index_of_intersection]) > 1:
                        
                        temp_strings_intersection_list[index_of_intersection]+=str(
                            max(temp_intersection_list[index_of_intersection], key=len))
                
                
                #Add the contents of the temp_strings_intersection_list to go_to_state
                go_to_state = ','.join(temp_strings_intersection_list)
                
                #Add the odds of this letter occurring to the proper spot of the to_return_df
                to_return_df.at[state, go_to_state]+=odds_dict[letter]

    return to_return_df

### Perform the N-player game by seeing the probability of string1 or string2 occurring first

**input**:  
*n_player_df*: The transition matrix derived from the above function 
*num_keystrokes*: How many keystrokes will the game go until?

**output**:  
*Pt*: Probability distribution of the N-player transition matrix after num_keystrokes_letters

In [ ]:
# def n_player_game(n_player_df, num_keystrokes=10000):
    
#     #Get the np.array of the transition df 
#     PHat = n_player_df.values
    
#     #Get the intiial dist from the associated identity matrix of the same size as the transition df
#     identity = np.identity(n_player_df.shape[0])
#     init_dist = identity[0]

    
#     #Phat^num_keystrokes: Intermediate step for the probability distribution calculation
#     PHat_exp = np.linalg.matrix_power(PHat, num_keystrokes)
    
#     #Get the probability distribution
#     Pt = init_dist @ PHat_exp
        
#     #Format the printing of this
#     for state in list(n_player_df.index):
#         print(f'Pr(Being in state {state}: {Pt[list(n_player_df.index).index(state)]})')
    
#     #Return prob. dist.
#     return Pt

In [ ]:
def n_player_game(n_player_df, strings_list, num_keystrokes=10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000):
    #Get the np.array of the transition df 
    PHat = n_player_df.values
    
    #Get the intiial dist from the associated identity matrix of the same size as the transition df
    identity = np.identity(n_player_df.shape[0])
    init_dist = identity[0]

    
    #Phat^num_keystrokes: Intermediate step for the probability distribution calculation
    PHat_exp = np.linalg.matrix_power(PHat, num_keystrokes)
    
    #Get the probability distribution
    Pt = init_dist @ PHat_exp
    
    #Create empty dict that will be filled. For a better formatted prob dist
    Pt_dict = {}
    
    #Fill the dict
    for state in list(n_player_df.index):
        Pt_dict[state] = Pt[list(n_player_df.index).index(state)]
    
    #Create empty list of the prob that each player wins
    prob_players_win = [0]*len(strings_list)
    
    
    #Fill the list by iterating thru each state...
    for key in Pt_dict.keys():
        
        #If the prob is not 0, we see which player wins corresponding to the state
        if Pt_dict[key] != 0:
            key_split = key.split(',')
            
            players_that_win = [i for i, x in enumerate(strings_list) if x in key_split]
            
            #Add this prob to the prob of them winning.
            for player in players_that_win:
                prob_players_win[player]+=Pt_dict[key]
                    
    #Format the printing of these probs
    
    #counter
    i = 1
    for player_prob in prob_players_win:
        print(f'Probability Player {i} winning: {round(player_prob, 15)}')
        i+=1
    
    
    return

# Function to Run Code

This function has no inputs or outputs, but does print all results to the screen.

In [ ]:
def run_code():
    
    #This is set equal to true when we get a valid input for the game being played
    valid_game_input = False
    
    while not valid_game_input:
        game_to_play = input("Please choose a game to play. Please press:\n'1' for the average hitting time of a "
                             "target string and a PMF of that hitting time\n'2' for the N-Player Game. ")
        if game_to_play not in ['1', '2']:
            print('Invalid input. Please try again.\n')
        else:
            valid_game_input = True
    
    #We now need to get the input for the target string, alphabet, odds, etc...
    alphabet_size = prompt_for_alphabet()
    
    #Convert this into an alphabet. It will be needed
    alphabet = ''.join(list(string.ascii_uppercase)[0:alphabet_size])
    
    #Get the odds dict
    odds_dict = specify_odds(alphabet_size)
    
    #If we are operating under the 1 target string case...
    if game_to_play == '1':
        
        #Prompt for the target string
        target_string = prompt_for_string(alphabet_size)
        
        #Get & print the avg hitting time
        avg_hitting_time = calculate_average_time(alphabet_size, target_string, odds_dict)
        print(f'Average Hitting Time of {target_string}: {avg_hitting_time} random keystrokes')
        
        #Get and print hitting time PMF
        
        #Intermediate step: Get the transition matrix
        transition_matrix = build_transition_matrix_one_string(target_string, alphabet, odds_dict)
        
        #Get the hitting time PMF dict
        hitting_time_PMF_dict = calc_hitting_time_PMF(transition_matrix, target_string, stopping_prob=.00001)
        
        #Print the results
        print(f'Hitting Time PMF:\n')
        for key, value in hitting_time_PMF_dict.items():
            print(key, ': ', value)
    
    #If we are operating under the 2 target string case...
    else:
        
        #We need to init. an empty list of target strings that we will fill
        string_list = []
        
        #We also want this to be a while loop, so we need a stopping condition
        done_entering_strings = False
        
        #Use while loop to ask for strings
        while not done_entering_strings:
            
            #Ask for string
            string_to_add = prompt_for_string(alphabet_size)
            
            #Add it to the list
            string_list.append(string_to_add)
            
            #See if the user is done entering strings
            if len(string_list) >= 2:
                
                #Check for valid input for the Are We Done Prompt
                valid_input_are_we_done = False
                
                while not valid_input_are_we_done:
                    
                    are_we_done = input('Would you like to input any more strings? Enter Y/N: ')
                    if are_we_done not in ['Y','N','n','y']:
                        print('Invalid Input. Please try again.\n')
                        
                    elif are_we_done in ['N','n']:
                        valid_input_are_we_done = True
                        done_entering_strings = True
                    else:
                        valid_input_are_we_done = True
                        
        #Intermediate steps: Get valid states and df and trans. matrix
        valid_states = get_all_valid_states_n_players(string_list)
        states_df = initialize_df(valid_states)
        trans_df = make_n_player_transition_df(string_list, alphabet, odds_dict, states_df)
        
        #Play the N Player Game
        n_player_game(trans_df, string_list)
        
    return

In [ ]:
run_code()

In [ ]:
get_all_valid_states_n_players(['a', 'ababba'])

In [22]:
#Get a plot for poster


import plotly.io as pio
import plotly.express as px

tmat = build_transition_matrix_one_string('ABC', 'ABC', {'A': 1/3, 'B': 1/3, 'C': 1/3})
pmf = calc_hitting_time_PMF(tmat, 'ABC')


list(pmf.values())[:50]

# Create PMF plot
fig = px.bar(x=list(range(0, len(list(pmf.values())[:39])+1)), y=list(list(pmf.values())[:40]), labels={'x': 'Number of Keystrokes', 'y':'<b>Probability'})
fig.update_layout(title={
        'text': "<b>PMF of Hitting Time of String 'ABC'",
        'x':0.5,  # Title centered
        'font':dict(
            family="Garamond",  # Garamond font
            size=36,  # Font size
            color="black")},
                 
        font=dict(
        family="Garamond",  # Garamond font for all text
        size=24,  # Default font size
        color="black"  # Default font color
    ), 
                  xaxis=dict(
        tickvals=[0, 2,3,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40]),
                 yaxis=dict(tickvals=[.005, .01, .015, .02,.025,.03,.035,.04]))


fig.update_traces(marker_color='#17375E')  # Use hex color value

fig.update_layout(
    width=1200,  # Width of the output image in pixels
    height=400  # Height of the output image in pixels
)


fig.show()
pio.write_image(fig, 'plot.svg')

In [ ]:
pip install -U kaleido

In [ ]:
calculate_average_time(3, 'ABA', {'A': 1/3, 'B': 1/3, 'C': 1/3})